In [1]:
import random
import math
import numpy as np
import vrplib
import os

# Загрузка данных CVRP из файла формата TSPLIB-Style (пример)
def load_vrp(filename):
    with open(filename) as f:
        # примитивный парсинг — нужно доработать для всех типов данных
        lines = f.readlines()
    coords = []
    demands = []
    capacity = 0
    reading_nodes = False
    demand_section = False
    for line in lines:
        if 'CAPACITY' in line:
            capacity = int(line.split()[-1])
        if 'NODE_COORD_SECTION' in line:
            reading_nodes = True
            continue
        if reading_nodes:
            if line.strip() == 'DEMAND_SECTION':
                reading_nodes = False
                demand_section = True
                continue
            parts = line.strip().split()
            if len(parts) >= 3:
                coords.append((float(parts[1]), float(parts[2])))
        elif demand_section:
            if line.strip() == 'DEPOT_SECTION':
                demand_section = False
                continue
            parts = line.strip().split()
            if len(parts) >= 2:
                demands.append(int(parts[1]))
    return coords, demands, capacity

# Расчет евклидова расстояния
def euclidean(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

# Создаем матрицу расстояний
def create_distance_matrix(coords):
    n = len(coords)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i,j] = euclidean(coords[i], coords[j])
    return dist

# Функция оценки маршрута с учётом грузоподъемности
def decode_routes(chromosome, demands, capacity):
    routes = []
    current_route = []
    current_load = 0
    for customer in chromosome:
        d = demands[customer]
        if current_load + d > capacity:
            routes.append(current_route)
            current_route = [customer]
            current_load = d
        else:
            current_route.append(customer)
            current_load += d
    if current_route:
        routes.append(current_route)
    return routes

def route_cost(route, dist_matrix):
    if not route:
        return 0
    cost = dist_matrix[0][route[0]]  # из депо к первому клиенту
    for i in range(len(route)-1):
        cost += dist_matrix[route[i]][route[i+1]]
    cost += dist_matrix[route[-1]][0]  # назад к депо
    return cost

def total_cost(chromosome, dist_matrix, demands, capacity):
    routes = decode_routes(chromosome, demands, capacity)
    return sum(route_cost(r, dist_matrix) for r in routes)

# Генетический алгоритм
def GA_CVRP(dist_matrix, demands, capacity, population_size=950, generations=1000, mutation_rate=0.9, no_improve_limit=90):
    n_customers = len(demands) - 1
    customers = list(range(1, n_customers+1))  # клиенты без депо
    # Создаем начальную популяцию
    population = [random.sample(customers, n_customers) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')
    no_improve_count = 0

    for gen in range(generations):
        # Оценка
        population_cost = [(indiv, total_cost(indiv, dist_matrix, demands, capacity)) for indiv in population]
        population_cost.sort(key=lambda x: x[1])

        if population_cost[0][1] < best_cost:
            best_cost = population_cost[0][1]
            best_solution = population_cost[0][0]
            no_improve_count = 0
        else:
            no_improve_count += 1
        if no_improve_count >= no_improve_limit:
            # print(f"Остановка: нет улучшений {no_improve_limit} поколений")
            break

        # Селекция – отбор лучших 20%
        cutoff = population_size // 5
        selected = [ind for ind, cost in population_cost[:cutoff]]

        # Создаем новую популяцию
        new_population = selected.copy()

        while len(new_population) < population_size:
            p1, p2 = random.sample(selected, 2)
            # Скрешивание Ordered Crossover (OX)
            child = ordered_crossover(p1, p2)
            # Мутация
            if random.random() < mutation_rate:
                child = swap_mutation(child)
            new_population.append(child)
        population = new_population

        # if gen % 50 == 0:
        #     print(f"Поколение {gen}, лучший результат {best_cost:.2f}")

    return best_solution, best_cost

def ordered_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None]*size
    child[a:b] = p1[a:b]
    fill_pos = b
    for i in range(size):
        gene = p2[(b + i) % size]
        if gene not in child:
            if fill_pos >= size:
                fill_pos = 0
            child[fill_pos] = gene
            fill_pos += 1
    return child

def swap_mutation(chromosome):
    a, b = random.sample(range(len(chromosome)), 2)
    chromosome[a], chromosome[b] = chromosome[b], chromosome[a]
    return chromosome

In [ ]:
if __name__ == '__main__':
    folder = 'C:/Users/user/Downloads/testData/'
    paths = os.listdir('C:/Users/zhengs/Downloads/testData/')
    for path in paths:
        print(path)
        coords, demands, capacity = load_vrp(folder+path)
        dist_matrix = create_distance_matrix(coords)
        diffs = []
    
        for i in range(1, 10):
                
            best_sol, best_cost = GA_CVRP(dist_matrix, demands, capacity, 1250, 1200, 0.9, 120)
            instance = vrplib.read_instance(folder+path)
            optimal_length = int(instance['comment'].split(',')[2].split()[-1][:-1])
            diff = 100*best_cost/optimal_length - 100
            diffs.append(diff)
        print('pop_size=',1250, 'diff=',sum(diffs)/len(diffs))